In [1]:
# dependencies
import torch
from torchvision.models import resnet18
import torch.optim as optim
from torch import nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

#################### Your Code ####################
ROOT_PATH=globals()['_dh'][0]  # Jupyter Notebook hack to get location of notebook file, https://stackoverflow.com/a/53958599
###################################################

BATCH_SIZE=256

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = CIFAR10(root=ROOT_PATH, download=True, train=True, transform=transform)
eval_dataset = CIFAR10(root=ROOT_PATH, train=False, transform=transform)

train_data_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
eval_data_loader = DataLoader(dataset=eval_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=False)


#################### Your Code ####################
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = resnet18(weights = None)

###################################################


Files already downloaded and verified


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
model.to(device)
model.train()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
CHECKPOINT_SIZE = 100

for epoch in range(10):

    total_loss = 0.0
    for i, batch in enumerate(train_data_loader):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(images)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if i % CHECKPOINT_SIZE == CHECKPOINT_SIZE - 1:    # print every 2000 mini-batches
            print(f'\r[{epoch + 1}, {i + 1:5d}] loss: {total_loss / CHECKPOINT_SIZE:.3f}')
            total_loss = 0.0

print('Finished Training')

[1,   100] loss: 2.690
[2,   100] loss: 1.421
[3,   100] loss: 1.231
[4,   100] loss: 1.065
[5,   100] loss: 0.923
[6,   100] loss: 0.785
[7,   100] loss: 0.653
[8,   100] loss: 0.506
[9,   100] loss: 0.382
[10,   100] loss: 0.259
Finished Training
